In [1]:
import math, random

# Função conversora de binário para decimal
def convBinDec(b):
    d = 0
    for i in range(len(b)):
        d += int(b[i]) * (2 ** ((len(b) - 1) - i))
    return d

# Função conversora de decimal para binário
def convDecBin(d):
    b1 = ""
    b2 = ""
    div = int(d)
    if d == 0:
        return "00000"
    else:
        while div != 1:
            b1 += str(div % 2)
            div = int(div/2)
        b1 += "1"
        for i in range(len(b1)-1, -1, -1):
            b2 += b1[i]
        while len(b2) < 5:
            b2 = "0" + b2
        return b2

# Função para obter as letras em hexadecimal
def convHexLetra(n):
    if n == 10:
        return "A"
    elif n == 11:
        return "B"
    elif n == 12:
        return "C"
    elif n == 13:
        return "D"
    elif n == 14:
        return "E"
    else: return "F"

# Função conversora de binário para hexadecimal
def convBinHex(b):
    h = ""
    while True: # Incrementa mais zeros no binário para formar agrupamentos de 4 bits
        if (len(b) % 4) == 0:
            break
        else:
            b = "0" + b
        
    for i in range(0, len(b), 4):
        num = int(b[i])*8 + int(b[i+1])*4 + int(b[i+2])*2 + int(b[i+3])*1
        if num > 9:
            h += str(convHexLetra(num))
        else: h += str(num)
    
    return h

# Função conversora de hexadecimal para binário
def convHexBin(h):
    
    b = ""
    
    for i in range(len(h)):
        if h[i] == "0": b += "0000"
        elif h[i] == "1": b += "0001"
        elif h[i] == "2": b += "0010"
        elif h[i] == "3": b += "0011"
        elif h[i] == "4": b += "0100"
        elif h[i] == "5": b += "0101"
        elif h[i] == "6": b += "0110"
        elif h[i] == "7": b += "0111"
        elif h[i] == "8": b += "1000"
        elif h[i] == "9": b += "1001"
        elif h[i] == "A": b += "1010"
        elif h[i] == "B": b += "1011"
        elif h[i] == "C": b += "1100"
        elif h[i] == "D": b += "1101"
        elif h[i] == "E": b += "1110"
        elif h[i] == "F": b += "1111"
        
    return str(b)

# Função do somador de 1 bit
def somaBin(b1, b2):
    global sobra
    if sobra == "1":
        if (b1 == "1" and b2 == "0") or (b1 == "0" and b2 == "1"):
            sobra = "1"
            return "0"
        elif b1 == "0" and b2 == "0":
            sobra = "0"
            return "1"
        elif b1 == "1" and b2 == "1":
            sobra = "1"
            return "1"
    else:
        if (b1 == "1" and b2 == "0") or (b1 == "0" and b2 == "1"):
            return "1"
        elif b1 == "0" and b2 == "0":
            return "0"
        elif b1 == "1" and b2 == "1":
            sobra = "1"
            return "0"

# Função do subtrator de 12 bits
def subtraiBin(b1, b2):
    print(b1 + " - " + b2)

    complemento1 = ""
    for i in range(len(b2)): # Realiza o complemento de 1
        if b2[i] == "0":
            complemento1 += "1"
        else:
            complemento1 += "0"
    print("\nBinário: "+b2+"\nComplemento de 1: "+complemento1)

    um = "1"
    while len(complemento1) > len(um): # Cria o binário 1 do tamanho do complemento de 1
        um = "0" + um

    global sobra
    sobra = "0"
    complemento2 = ""
    for i in range(len(complemento1)-1, -1, -1): #  Realiza o complemento de 2
        complemento2 = somaBin(complemento1[i], um[i]) + complemento2 # Chamada do somador de 1 bit
    print("Complemento de 2: "+complemento1+" + 1 = "+complemento2)

    sobra = "0"
    sub = ""
    for i in range(len(complemento2)-1, -1, -1): # Realiza a soma do primeiro binário com o complemento de 2 do segundo
        sub = somaBin(b1[i], complemento2[i]) + sub # Chamada do somador de 1 bit
    print("\nSoma final: \n"+b1 + " + " + complemento2+ " = "+sub)
    return sub

# Procedimento que quebra o endereço em Tag, Conjunto e Célula
def quebraEndereco(e):
    global endTag
    global endConjunto
    global endCelula
    endTag = ""
    endConjunto = ""
    endCelula = ""
    
    if len(e) > 5: # Verifica se o endereço possui mais que 5 bits
        endereco = ""
        for i in range((len(e)-1), 2, -1): # Percorre o endereço do bit menos significativo até o quinto bit
            endereco = e[i] + endereco # Guarda apenas 5 bits
    else: endereco = e
    
    for i in range(len(endereco)):
        if i >= 0 and i < tagBits:
            endTag = endTag + endereco[i] # Separa os bits da tag
        elif i >= tagBits and i < (tagBits + conjuntoBits):
            endConjunto = endConjunto + endereco[i] # Separa os bits do conjunto
        elif i >= (tagBits + conjuntoBits):
            endCelula = endCelula + endereco[i] # Separa os bits da celula/byte
    print("\nFormato de endereço da cache com mapeamento associativo por conjunto de",LpConj,":"+
              "\n| Tag      |",endTag,
              "\n| Conjunto |",endConjunto,
              "\n| Célula   |",endCelula)

# Função para buscar a tag na cache
def buscaTag(c):
    global iBusca # Índice de busca, se der Hit, ela vai estar com o índice em que a tag foi encontrada
    busca = False
    for iBusca in range((c * LpConj), ((c * LpConj) + LpConj)): #Laço que percorre as linhas do conjunto
        if endTag == cache[iBusca][len(cache[iBusca])-1]: # Verifica se a tag está na linha
            busca = True
            break
    return busca
    
# Procedimento para atualizar determinada linha da cache
def atualizaCache(c):
    global endTag
    global endConjunto
    global linha
    bloco = convBinDec(endTag+endConjunto) #Bloco da MP desejado
    print("\nBloco buscado na MP:", endTag+endConjunto)
    linha = random.randrange((c * LpConj), ((c * LpConj) + LpConj)) #Linha aleatória no conjunto
    print("Linha atualizada na Cache:", linha)
    j = 0
    for i in range((bloco * X), ((bloco * X) + X)): #Laço que percorre o bloco
        cache[linha][j] = mp[i] #Adiciona as células do bloco na linha da cache
        j += 1
    cache[linha][(len(cache[linha])-1)] = endTag #O último item da lista do cache[linha] recebe a tag

# Procedimento para saída dos dados da linha da cache
def exibeLinhaCache(l):
    saida = "| "+cache[l][len(cache[l])-1] + " | "
    for j in range(len(cache[l])-2, -1, -1):
        saida = saida + cache[l][j] + " | " + convBinHex(cache[l][j]) + " | "
    print(saida)

# Procedimento que lista a MP
def exibeMP():
    print("\nMemória Principal\nEnd   |     Bin      | Hex |\n----------------------------")
    for i in range(Np):
        print(convBinHex(convDecBin(i))+"    | "+mp[i]+" | "+convBinHex(mp[i])+" |") # Imprime a MP com as instruções e dados inseridos

# Procedimento que faz a operação de escrita na cache e na MP
def realizaEscrita(e,d):
    print("\n>>> Escrita <<<")
    # Quebra do endereço
    quebraEndereco(e) # Chamada do procedimento  
        
    #Escrita na Cache
    conjunto = convBinDec(endConjunto) #Conversão do bit do binário para decimal
    print("\nEscrita no conjunto",conjunto,"da Cache:")
    
    global linha
    if(buscaTag(conjunto) == True): # Atribui a linha dependendo da tag estar ou não no conjunto, chamando a função de busca
        print("Bloco "+endTag+endConjunto+" encontrado na Cache\nLinha ",iBusca)
        linha = iBusca # A linha recebe a posição da linha na cache em que a tag foi encontrada
    else:
        print("Bloco "+endTag+endConjunto+" não encontrado na Cache")
        atualizaCache(conjunto) # Chamada do procedimento
        
    cache[linha][convBinDec(endCelula)] = d # A célula atualizada recebe o dado novo
    print("\nLinha", linha, "<--- " + d)
    print("| Tag |      Célula 1      |      Célula 0      |")
    exibeLinhaCache(linha) # Chamada do procedimento
        
    #Escrita na Memória Principal
    print("\nEscrita na Memória Principal\nMP <--- Cache")
    mp[convBinDec(e)] = cache[linha][convBinDec(endCelula)] # Cache envia célula para MP conforme o endereço inserido
    print("Endereço " + convBinHex(e) + " <--- " +cache[linha][convBinDec(endCelula)]+
          " | "+convBinHex(cache[linha][convBinDec(endCelula)]))
    print("")

# Função que faz a operação de leitura na cache e na MP
def realizaLeitura(e):
    print("\n>>> Leitura <<<")
    # Quebra do endereço
    quebraEndereco(e) # Chamada do procedimento  

    # Busca da tag na cache
    print("\nAcesso ao endereço na Memória Cache para Leitura")
    conjunto = convBinDec(endConjunto) #Conversão do bit do binário para decimal
    print("Busca no Conjunto",conjunto)
    
    global linha
    if buscaTag(conjunto) == True:  #Se a busca der Hit, chamando a função que faz a busca
        print("Resultado: Hit\nLinha ",iBusca)
        linha = iBusca # A linha recebe a posição da linha na cache em que a tag foi encontrada
        print("\nInformação na Cache:")
    else: #Se a busca der Miss
        print("Resultado: Miss\nO bloco será buscado na Memória Principal")
        atualizaCache(conjunto) # Chamada do procedimento
    
    print("| Tag |      Célula 1      |      Célula 0      |")
    exibeLinhaCache(linha) # Chamada do procedimento
    return cache[linha][convBinDec(endCelula)]      
        
# Função que retorna a instrução de determinado endereço
def separaInstrucao():
    global RI
    inst = ""
    for i in range(4):
        inst += RI[i]  # String inst recebe os bits da instrução
    return inst

# Função que retorna o endereço de um operando
def separaOperando():
    global RI
    op = ""
    for i in range((len(RI)-1),(len(RI)-6),-1): # Percorre os últimos 5 bits
        op = RI[i] + op # String op recebe os bits do endereço do operando
    return op

# Procedimento que verifica qual é a instrução
def executaInstrucao(instrucao):     
    global RDM
    global REM
    global R0
    global PC
    global flag
    
    print("\nUnidade Aritmética e Lógica - ULA:\nInstrução "+convBinHex(operacao))
    
    if instrucao == "0001": # 0001 - LDA Op (Load) - R0 <- (Op)
        print("LDA Op. (Load): R0 <- (Op.)")
        REM = separaOperando() # REM recebe endereço do operando obtido pela função
        RDM = realizaLeitura(REM)
        R0 = RDM # R0 recebe o dado do endereço do operando
        print("\nLoad  --->  End "+convBinHex(REM)+" | "+R0+" | "+convBinHex(R0)+" |\n"+
              "R0  <---  "+convBinHex(R0))
    
    elif instrucao == "0010": # 0010 - STR Op (Store) - (Op) <- R0
        print("STR Op. (Store): (Op.) <- R0")
        REM = separaOperando() # REM recebe endereço do operando obtido pela função
        RDM = R0 # RDM recebe o valor de R0
        realizaEscrita(REM, RDM) # O valor de R0 é armazenado no endereço da MP
        print("\nSTR Op. (Store): End | "+REM+" | "+convBinHex(REM)+" | <---  R0 | "+R0+" | "+convBinHex(R0))
        
    elif instrucao == "0011": # 0011 - ADD Op - R0 <- R0 + (Op)
        print("ADD Op. (Soma): R0 <- R0 + (Op)")
        REM = separaOperando() # REM recebe endereço do operando obtido pela função
        RDM = realizaLeitura(REM)
        
        print("\nADD Op.  --->  R0 | "+R0+" | "+convBinHex(R0)+
                " |  +  | "+RDM+" | "+convBinHex(RDM)+" |")
        global sobra
        sobra = "0"
        soma = ""
        for i in range(len(R0)-1, -1, -1): # Realiza a soma bit por bit
            soma = somaBin(R0[i], RDM[i]) + soma

        print("\nSoma: \n"+R0 + " + " + RDM+ " = "+ soma)
        R0 = soma # R0 recebe a soma resultado da função
        print("R0  <---  | "+R0+" | "+convBinHex(R0)+" |")
        
    elif instrucao == "0100": # 0100 - Sub Op - R0 <- R0 - (Op)
        print("SUB Op. (Subtração): R0 <- R0 - (Op.)")
        REM = separaOperando() # REM recebe endereço do operando obtido pela função
        RDM = realizaLeitura(REM)
        print("\nSUB Op.  --->  R0 | "+R0+" | "+convBinHex(R0)+
                " |  -  | "+RDM+" | "+convBinHex(RDM)+" |")
        if convBinDec(RDM) > convBinDec(R0):
            flag = 1 # Indica subtração resultará em um número negativo
        else:
            flag = 0 # Indica subtração resultará em um número positivo
        R0 = subtraiBin(R0, RDM) # R0 recebe a subtração de R0 pelo dado do endereço do operando
        print("R0  <---  | "+R0+" | "+convBinHex(R0)+" |")
    
    elif instrucao == "0101": # 0101 - JZ Op - Se R0 = 0, então: PC <- Op
        print("JZ Op.: Se R0 = 0 ---> Jump: Op. ---> PC")
        if R0 == convHexBin("000"): # Se o R0 for zero
            PC = separaOperando() # PC recebe endereço do operando obtido pela função para realizar o salto
            print("Jump realizado\nPC <---  End | "+PC+" | "+convBinHex(PC)+" |")
        else:print("Jump não realizado")
    
    elif instrucao == "0110": # 0110 - JP Op - Se R0 > 0, então: PC <- Op
        print("JP Op.: Se R0 > 0 ---> Jump: Op. ---> PC")
        if flag == 0: # Se o R0 for positivo
            PC = separaOperando() # PC recebe endereço do operando obtido pela função para realizar o salto
            print("Jump realizado\nPC <---  End | "+PC+" | "+convBinHex(PC)+" |")
        else:print("Jump não realizado")
    
    elif instrucao == "0111": # 0111 - JN Op - Se R0 < 0, então: PC <- Op
        print("JN Op.: Se R0 < 0 ---> Jump: Op. ---> PC")
        if flag == 1: # Se o R0 for negativo
            PC = separaOperando() # PC recebe endereço do operando obtido pela função para realizar o salto
            print("Jump realizado\nPC <---  End | "+PC+" | "+convBinHex(PC)+" |")
        else:print("Jump não realizado")
    
    elif instrucao == "1000": # 1000 - JMP Op - PC <- Op
        print("JMP Op. (Jump): PC <- Op.")
        PC = separaOperando() # PC recebe endereço do operando obtido pela função para realizar o salto
        print("Jump realizado\nPC <---  End | "+PC+" | "+convBinHex(PC)+" |")
            
    elif instrucao == "1001": # 1001 - GET Op - Ler dado da porta de entrada e armazená-lo em (Op)
        print("GET Op.: Ler dado da porta de entrada e armazená-lo em (Op.)")
        REM = separaOperando() # REM recebe endereço do operando obtido pela função
        # Entrada de dados para escrita
        while True:
            try:
                entrada = input("Digite o dado de 12 bits: ")
                # Validação da entrada
                for i in range(len(entrada)):
                    if entrada[i] != "0" and entrada[i] != "1": # Verifica se o dado possui apenas bits 0 e 1
                        raise Exception
                        break
                if len(entrada) != 12: # Verifica se o dado possui 12 bits
                    raise Exception
                break
            except Exception: # Caso o dado não seja de bits de 0 e 1 e não tenha 8 bits
                print("Dado inválido!")
        
        realizaEscrita(REM, entrada)
    
    elif instrucao == "1010": # 1010 - PRT Op - Colocar na porta referente à impressora o valor armazenado em (Op)
        print("PRT Op. (Print)")
        REM = separaOperando() # REM recebe endereço do operando obtido pela função
        RDM = realizaLeitura(REM)
        print("\n>>> Exibido na Tela <<<\nEnd "+convBinHex(REM)+"    | "+RDM+" | "+convBinHex(RDM)+" |")

# Procedimento que realiza os ciclos das instruções na execução do programa
def executaPrograma():
    # Variáveis do processador
    global RDM # Registrador de Dados da Memória
    global REM # Registrador de Endereços da Memória
    global RI # Registrador de Instruções
    global PC # CI
    global R0 # Registrador de Dados
    global operacao # Operacão da instrução
    RDM = ""
    REM = ""
    RI = ""
    PC = ""
    R0 = ""
    operacao = ""

    # Execução do programa
    print("\n>>> Programa iniciado <<<")

    PC = "00000" # PC inicia com a primeira instrução
    ciclo = 0
    while True: # Roda todas as intruções
        ciclo += 1
        print("\nCiclo",(ciclo))
        REM = PC # REM recebe o endereço da instrução
        print("REM <--- PC "+PC)
        PC = convDecBin(convBinDec(PC) + 1) # PC é incrementado para a próxima instrução
        print("Próxima instrução: PC <--- End | "+PC+" | "+convBinHex(PC)+" |")
        RDM = realizaLeitura(REM) # RDM recebe a instrução do endereço no REM
        print("\nRDM <--- End "+convBinHex(REM)+"    | "+RDM+" | "+convBinHex(RDM)+" |")
        RI = RDM # A instrução é guardada no RI
        print("RI <--- RDM | "+ RDM+" | "+convBinHex(RDM)+" |")
        operacao = separaInstrucao() # Chamada da função que separa a instrução
        if operacao == "0000":  # 0000 - HLT (Halt) - Para a execução do programa
            print("HLT (Halt): Para a execução do programa")
            break
        else:
            print("Unidade de Controle - UC:\nInstrução | "+operacao+" |")
            executaInstrucao(operacao) # Chamada da função que executa a instrução do RI

    print("\n>>> Programa encerrado <<<")

# Memória Principal
mp = [] # Array da Memória Principal
Np = 32  # Capacidade de células da MP

# Memória Cache
cache = [] # Array da Memória Cache
X = 2  # Quantidade de células por bloco da MP/largura das linhas da cache
Ncache = 8  # Capacidade da cache
L = 4  # Quantidade de linhas da cache
LpConj = 2  # Quantidade de linha por conjunto

# Quantidade de blocos
B = int(Np / X)  # B = 16
# Quantidade de linhas
L = int(Ncache / X)  # L = 4
# Quantidade de conjuntos
C = int(L / LpConj)  # C = 2
# Quantidade bloco da MP por conjunto
BpConj = int(B / C)  # BpConj = 8
# Quantidade de bits da Tag
tagBits = int(math.log2(BpConj))  # tagBits = 3 bits
# Quantidade de bits do Conjunto
conjuntoBits = int(math.log2(C))  # conjuntoBits = 1 bit
# Quantidade de bits da Célula
celulaBits = int(math.log2(X))  # celulaBits = 1 bit

# Dados da Memória Cache: Célula 0 | Célula 1 | Tag, inserção das instruções na cache
cache.append(["000000000000", "000000000000", "000"])  # Linha 0 -- Conjunto 0 (Endereços 00000 e 00001)
cache.append(["000000000000", "000000000000", "001"])  # Linha 1 -- Conjunto 0 (Endereços 00100 e 00101)
cache.append(["000000000000", "000000000000", "000"])  # Linha 2 -- Conjunto 1 (Endereços 00010 e 00011)
cache.append(["000000000000", "000000000000", "001"])  # Linha 3 -- Conjunto 1 (Endereços 00110 e 00111)

print("\nMemória Cache\n| Tag |      Célula 1      |      Célula 0      |\n-------------------------------------------------")
for i in range(len(cache)): # Formatação da saída das linhas da cache e listagem
    exibeLinhaCache(i) # Chamada do procedimento

# Dados da Memória Principal
for i in range(Np):
    mp.append("000000000000") #MP inicialmente zerada

exibeMP() # Chamada do procedimento que lista a MP

'''
Adaptação das instruções para Memória de 32 células

Programa 1: Maior entre dois números
Instruções:
End | Dado
00  | 1B4 --> 11A
01  | 4B5 --> 41B
02  | 705
03  | AB4 --> A1A
04  | 806
05  | AB5 --> A1B
06  | 000
Dados:
End | Dado     End | Dado
B4  | 2B3  --> 1A  | 2B3 (escolha do usuário)
B5  | 1A3  --> 1B  | 1A3 (escolha do usuário)

Programa 2: Contagem regressiva
Instruções:
End | Dado
00  | AB5 --> A1B
01  | 1B5 --> 11B
02  | 4B4 --> 41A
03  | 2B5 --> 21B
04  | AB5 --> A1B
05  | 507
06  | 801
07  | 000
Dados:
End | Dado     End | Dado
B4  | 001  --> 1A  | 001
B5  | 010  --> 1B  | 010 (escolha do usuário)
'''
  
# Operações da Memória
while True:
    print("\nSIMULADOR DE MEMÓRIA\n"+
          "[M] Programa: Maior entre dois números\n"+
          "[C] Programa: Contagem regressiva\n"+
          "[W] Escrever\n[R] Ler\n[L] Listar")
    comando = input("Digite uma das opções ou qualquer tecla para parar: ") # Escolha das operações

    # PROGRAMA: MAIOR ENTRE DOIS NÚMEROS
    if comando == "M" or comando == "m":
        # Inserção das instruções e dados na MP e Cache para o Programa 1: Menor entre dois números
        instrucoes = ["11A","41B","705","A1A","806","A1B","000"] # Array com as instruções do programa 1
        
        for i in range(len(instrucoes)):
            # As instruções serão armazenadas do endereço 00000 ao 00110
            realizaEscrita(convDecBin(i), convHexBin(instrucoes[i])) 
            
        # Entrada de dados para escrita
        while True:
            try:
                dado = input("Primeiro valor:\nDigite o dado de 12 bits: ")
                # Validação da entrada
                for i in range(len(dado)):
                    if dado[i] != "0" and dado[i] != "1": # Verifica se o dado possui apenas bits 0 e 1
                        raise Exception
                        break
                if len(dado) != 12: # Verifica se o dado possui 12 bits
                    raise Exception
                break
            except Exception: # Caso o dado não seja de bits de 0 e 1 e não tenha 8 bits
                print("Dado inválido!")
        
        realizaEscrita(convHexBin("1A"), dado) # Escrita do bináio no endereço 1A
        
        while True:
            try:
                dado = input("Segundo valor:\nDigite o dado de 12 bits: ")
                # Validação da entrada
                for i in range(len(dado)):
                    if dado[i] != "0" and dado[i] != "1": # Verifica se o dado possui apenas bits 0 e 1
                        raise Exception
                        break
                if len(dado) != 12: # Verifica se o dado possui 12 bits
                    raise Exception
                break
            except Exception: # Caso o dado não seja de bits de 0 e 1 e não tenha 8 bits
                print("Dado inválido!")
        
        realizaEscrita(convHexBin("1B"), dado) # Escrita do bináio no endereço 1B
        
        print("\nMemória Cache\n| Tag |      Célula 1      |      Célula 0      |\n-------------------------------------------------")
        for i in range(len(cache)): # Formatação da saída das linhas da cache e listagem
            exibeLinhaCache(i) # Chamada do procedimento
        exibeMP()
        executaPrograma() # Chamada do procedimento que realiza a execução dos programas e das instruções armazenadas
    
    #PROGRAMA: CONTAGEM REGRESSIVA
    elif comando == "C" or comando == "c":
        # Inserção das instruções e dados na MP e Cache para o Programa 2: Contagem regressiva
        instrucoes = ["A1B","11B","41A","21B","A1B","507","801","000"] # Array com as instruções do programa 2
        
        for i in range(len(instrucoes)):
            # As instruções serão armazenadas do endereço 00000 ao 00111
            realizaEscrita(convDecBin(i), convHexBin(instrucoes[i])) 
        
        realizaEscrita(convHexBin("1A"), convHexBin("001")) # Insere o dado 001 no endereço 1A
        
        # Entrada de dados para escrita
        while True:
            try:
                dado = input("Limite da contagem:\nDigite o dado de 12 bits: ")
                # Validação da entrada
                for i in range(len(dado)):
                    if dado[i] != "0" and dado[i] != "1": # Verifica se o dado possui apenas bits 0 e 1
                        raise Exception
                        break
                if len(dado) != 12: # Verifica se o dado possui 12 bits
                    raise Exception
                break
            except Exception: # Caso o dado não seja de bits de 0 e 1 e não tenha 8 bits
                print("Dado inválido!")
        
        realizaEscrita(convHexBin("1B"), dado) # Escrita do bináio no endereço 1B
        
        print("\nMemória Cache\n| Tag |      Célula 1      |      Célula 0      |\n-------------------------------------------------")
        for i in range(len(cache)): # Formatação da saída das linhas da cache e listagem
            exibeLinhaCache(i) # Chamada do procedimento
        exibeMP() # Chamada do procedimento que lista a MP
        executaPrograma() # Chamada do procedimento que realiza a execução dos programas e das instruções armazenadas
    
    # ESCRITA
    elif comando == "W" or comando == "w":
        # Entrada de endereço para escrita
        while True:
            try:
                endereco = input("Digite o endereço de 5 bits: ") # Verifica se o endereço possui apenas bits 0 e 1
                # Validação da entrada
                for i in range(len(endereco)):
                    if endereco[i] != "0" and endereco[i] != "1":
                        raise Exception
                        break
                if len(endereco) != 5: # Verifica se o endereço possui 5 bits
                    raise Exception
                break
            except Exception: # Caso o endereço não seja de bits de 0 e 1 e não tenha 5 bits
                print("Endereço inválido!")

        # Entrada de dados para escrita
        while True:
            try:
                dado = input("Digite o dado de 12 bits: ")
                # Validação da entrada
                for i in range(len(dado)):
                    if dado[i] != "0" and dado[i] != "1": # Verifica se o dado possui apenas bits 0 e 1
                        raise Exception
                        break
                if len(dado) != 12: # Verifica se o dado possui 12 bits
                    raise Exception
                break
            except Exception: # Caso o dado não seja de bits de 0 e 1 e não tenha 8 bits
                print("Dado inválido!")
        
        realizaEscrita(endereco, dado)

    # LEITURA
    elif comando == "R" or comando == "r":
        # Entrada de endereço para leitura
        while True:
            try:
                endereco = input("Digite o endereço de 5 bits: ") # Verifica se o endereço possui apenas bits 0 e 1
                # Validação da entrada
                for i in range(len(endereco)):
                    if endereco[i] != "0" and endereco[i] != "1":
                        raise Exception
                        break
                if len(endereco) != 5: # Verifica se o endereço possui 5 bits
                    raise Exception
                break
            except Exception: # Caso o endereço não seja de bits de 0 e 1 e não tenha 5 bits
                print("Endereço inválido!")
        
        leitura = realizaLeitura(endereco)
        print("\nEndereço: " + convBinHex(endereco) + 
          "\nDado: " + leitura +" | "+convBinHex(leitura))
        print("")  
            
    # LISTAGEM
    elif comando == "L" or comando == "l":
        print("\nMemória Cache:\n| Tag |      Célula 1      |      Célula 0      |\n-------------------------------------------------")
        for i in range(len(cache)): # Formatação da saída das linhas da cache e listagem
            exibeLinhaCache(i) # Chamada do procedimento
        
        exibeMP() #Listagem da MP
            
    # Encerramento
    else:
        print(">>> Computador desligando <<<")
        break



Memória Cache
| Tag |      Célula 1      |      Célula 0      |
-------------------------------------------------
| 000 | 000000000000 | 000 | 000000000000 | 000 | 
| 001 | 000000000000 | 000 | 000000000000 | 000 | 
| 000 | 000000000000 | 000 | 000000000000 | 000 | 
| 001 | 000000000000 | 000 | 000000000000 | 000 | 

Memória Principal
End   |     Bin      | Hex |
----------------------------
00    | 000000000000 | 000 |
01    | 000000000000 | 000 |
02    | 000000000000 | 000 |
03    | 000000000000 | 000 |
04    | 000000000000 | 000 |
05    | 000000000000 | 000 |
06    | 000000000000 | 000 |
07    | 000000000000 | 000 |
08    | 000000000000 | 000 |
09    | 000000000000 | 000 |
0A    | 000000000000 | 000 |
0B    | 000000000000 | 000 |
0C    | 000000000000 | 000 |
0D    | 000000000000 | 000 |
0E    | 000000000000 | 000 |
0F    | 000000000000 | 000 |
10    | 000000000000 | 000 |
11    | 000000000000 | 000 |
12    | 000000000000 | 000 |
13    | 000000000000 | 000 |
14    | 000000000000 | 0